# Computing Valuation Manually

We wish to compute $\textrm{ord}_P(f)$ where $P = (2, 4)$ and $f = y - 2x$.

$$E(\mathbb{F}_{11}) : y^2 = x^3 + 4x$$

Let $K[V] = K[x, y] / \langle y^2 - x^3 - 4x \rangle$ by the coordinate ring. $K(V)$ is the field of fractions for $K[V]$.

In [2]:
# Define our base polynomial ring over Z_11
K.<x, y> = GF(11)[]
# This is K(V)
S = K.quotient(y^2 - x^3 - 4*x).fraction_field()
S

Fraction Field of Quotient of Multivariate Polynomial Ring in x, y over Finite Field of size 11 by the ideal (-x^3 + y^2 - 4*x)

Because $P$ lies on the curve $E$, we can take the nontangent lines $x - 2, y - 4$ as a basis for the local curve. The intuition is that these lines describe the coordinate grid around $P$, and we can multiply them by any polynomial to get cosets of $K(V)$.

More formally we can see this by noting that:

$$(y - 4)(y + 4) = (x - 2)^3 - 5(x - 2)^2 - 6(x - 2)$$

So therefore any function on $E$ can be expressed in terms of $(x - 2)$ and $(y - 4)$.

In [3]:
X, Y = S(x), S(y)
(Y - 4)*(Y + 4) == (X - 2)^3 - 5*(X - 2)^2 - 6*(X - 2)

True

Begin by expressing $f = y - 2x$ in terms of this basis.
\begin{align*}
y &= y - 2x \\
  &= -2(x - 2) + 1(y - 4) \\
\mathbf{a} &= (-2, 1, 0) \\
\mathbf{b} &= (x - 2, y - 4, 1)
\end{align*}

In [4]:
# Components for f
a0, a1, a2 = -2, 1, 0
# Our basis
b0, b1, b2 = x - 2, y - 4, 1
a0*b0 + a1*b1 + a2*b2

-2*x + y

Using the identity above for $(y - 4)(y + 4)$, we can see that
\begin{align*}
(y - 4) &= \frac{(x - 2)^3 - 5*(x - 2)^2 - 6(x - 2)}{(y + 4)} \\
        &= (x - 2) \frac{(x - 2)^2 - 5*(x - 2) - 6}{(y + 4)}
\end{align*}
So we can know that $b_1 = (E_f / E_g) b_0$, and can make this substitution in $f$.
\begin{align*}
f &= -2 (x - 2) + 1 \cdot \frac{(x - 2)^3 - 5(x - 2)^2 - 6(x - 2)}{y + 4} \\
  &= \frac{1}{(y + 4)}[(x - 2)^3 - 5(x - 2)^2 - 6(x - 2) - 2(x - 2)(y + 4)]
\end{align*}

In [5]:
# and lets double check this
B0 = X - 2
Eg = Y + 4
(B0^3 - 5*B0^2 - 6*B0 - 2*B0*Eg)/Eg == Y - 2*X

True

Now lets quotient out $b_0 = (x - 2)$ from $f$ to get the first power of $k = 1$ for the uniformizer.
\begin{align*}
f &= \frac{(x + 2)^1}{(y + 4)}[(x - 2)^2 - 5(x - 2) - 6 - 2(y + 4)]
\end{align*}
Notice now the constant term is $- 6 - 2(y + 4)$ which should be a multiple of $y - 4$ if we can continue extracting $b_0$ from the expression.

In [6]:
-6 - 2*(y + 4)

-2*y - 3

In [7]:
-2*(y - 4)

-2*y - 3

Great so we continue. Instead of writing $-6 - 2(y + 4)$, lets instead write $-2(y - 4)$
\begin{align*}
f &= \frac{(x + 2)^1}{(y + 4)}[(x - 2)^2 - 5(x - 2) - 2(y - 4)] \\
  &= \frac{(x + 2)^1}{(y + 4)}[(x - 2)^2 - 5(x - 2) - 2\frac{(x - 2)^3 - 5(x - 2)^2 - 6(x - 2)}{y + 4}] \\
  &= \frac{(x + 2)^1}{(y + 4)^2}[(x - 2)^2(y + 4) - 5(x - 2)(y + 4) - 2((x - 2)^3 - 5(x - 2)^2 - 6(x - 2))] \\
  &= \frac{(x + 2)^2}{(y + 4)^2}[(x - 2)(y + 4) - 5(y + 4) - 2((x - 2)^2 - 5(x - 2) - 6)] \\
\end{align*}
So $k = 2$, and lets evaluate the constant terms.

In [8]:
-5*(y + 4) - 2*(-6)

-5*y + 3

In [9]:
-5*(y - 4)

-5*y - 2

Now there's a remainder left over so the expression terminates.

# Implementation Details

Several points of interest:

* We don't need to keep track of $g$ although it's done for completeness.
* For the uniformizer part, we just need to keep track of $k$.
* The inner expression can be optimized just by looking at the constant term when viewed from $(x - 2)$. Although we are not doing that.

In [10]:
b0, b1, b2

(x - 2, y - 4, 1)

In [11]:
# Decompose polynomial function into basis components
def decomp(f):
    a0, r = f.quo_rem(b0)
    a1, r = r.quo_rem(b1)
    a2, r = r.quo_rem(b2)
    return a0, a1, a2

f = y - 2*x
decomp(f)

(-2, 1, 0)

In [12]:
Px, Py = (2, 4)
EC_A = 4
# Calculate substitution polynomials
Ef = b0^2 + binomial(3,2)*Px*b0^1 + (3*Px^2 + EC_A)
Eg = (y + Py)
# Should be EC equation
Eg*b1 - Ef*b0

-x^3 + y^2 - 4*x

In [13]:
a0, a1, a2 = decomp(f)
g = 1

# Perform first reduction
a0 = Eg*a0 + Ef*a1
a1 = 0
g *= Eg
assert a1 == a2 == 0
a0, g

(x^2 + 2*x - 2*y, y + 4)

Set $k = 1$ since we have now factored out $(x - 2)$. Continue for second decomposition followed by reduction.

In [14]:
a0, a1, a2 = decomp(a0)
a0, a1, a2

(x + 4, -2, 0)

as expected the remainder a2 is zero

In [15]:
U = (X - 2)
F = (X + 4)*(X - 2) - 2*(Y - 4)
G = Y + 4
U^1 * F / G == Y - 2*X

True

In [16]:
# Second reduction
a0 = Eg*a0 + Ef*a1
a1 = 0
g *= Eg
assert a1 == a2 == 0
a0, g

(-2*x^2 + x*y + 4*y, y^2 - 3*y + 5)

Set $k = 2$. Now decompose and check remainder is zero again before performing reduction.

In [17]:
a0, a1, a2 = decomp(a0)
a0, a1, a2

(-2*x + y - 4, -5, 5)

Now the remainder is 5 so the algorithm stops. Our final valuation is $k = 2$.

In [18]:
# Convert our function to the function field K(V)
original_f = Y - 2*X
k = 2

f = a0*b0 + a1*b1 + a2*b2
fprime = b0^k * f/g
assert fprime == S(original_f)
assert g(Px, Py) != 0
assert f(Px, Py) != 0
assert b0(Px, Py) == 0


# Abstract Algebraic Method

See Knapp page 350

In [19]:
R.<x> = FunctionField(GF(11)); _.<Y> = R[]
K.<y> = R.extension(Y^2 - x^3 - 4*x)
P = (2, 4)
o = R.maximal_order()
O = K.maximal_order()
M = o.ideal(x - 2)
# if I = <x - 2> then (x - 2) in I^2 is false 
x - 2 in M, x - 2 in M^2

(True, False)

Also verify that the point is ordinary (nonsingular)

In [20]:
M.is_prime()

True

In [21]:
O.decomposition(M)

[(Ideal (x + 9, y + 4) of Maximal order of Function field in y defined by y^2 + 10*x^3 + 7*x,
  1,
  1),
 (Ideal (x + 9, y + 7) of Maximal order of Function field in y defined by y^2 + 10*x^3 + 7*x,
  1,
  1)]

In [22]:
# I can't seem to use M for f
# This is a workaround until I find a fix
M = O.ideal(x - 2, y - 4)

In [23]:
f = y - 2*x
f in M, f in M^2, f in M^3

(True, True, False)

In [24]:
d = [i for i in range(1, 10) if f in M^i]
max(d)

2

# Simple Method by Parameterizing

We parameterize our line $f$ so that $(x(0), y(0)) = P$

In [25]:
K.<t> = GF(11)[]
x = t + 2
# y - 2x = 0
# => y = 2x
y = 2*x
# Now just write equation in terms of parameterized x(t) and y(t)
y^2 - x^3 - 4*x

10*t^3 + 9*t^2

As observed we have a uniformizer $t$ with $k = 2$.

# Valuation with Places

Using sage valuation rings since local ring is a DVR

In [26]:
K.<x> = FunctionField(GF(11))
_.<Y> = K[]
L.<y> = K.extension(Y^2 - x^3 - 4*x)
L.places_finite()

[Place (x, y),
 Place (x + 3, y + 4),
 Place (x + 3, y + 7),
 Place (x + 5, y + 3),
 Place (x + 5, y + 8),
 Place (x + 7, y + 5),
 Place (x + 7, y + 6),
 Place (x + 9, y + 4),
 Place (x + 9, y + 7),
 Place (x + 10, y + 4),
 Place (x + 10, y + 7)]

We are interested in $P = (2, 4)$ so we want the place $(x - 2, x - 4) = (x + 9, y + 7)$.

In [27]:
# I don't know how to actually construct this yet
p = L.places_finite()[-3]
p

Place (x + 9, y + 7)

In [28]:
(y - 2*x).valuation(p)

2